In [1]:
import numpy as np
import typing
import enum


def create_weights(neurons: typing.Iterable[int]):
    np.random.seed(1)
    if len(neurons) < 2:
        raise Exception("At least two values (input layer & output layer) are required")
    for size in neurons:
        if size < 1:
            raise Exception("Minimum 1 neuron per layer")
    weights: list[np.ndarray[typing.Any, np.dtype[np.float64]]] = []
    for i in range(0, len(neurons)-1):
        weights.append(2 * np.random.random((neurons[i], neurons[i+1])) - 1)
    return weights


class ActivatorType(enum.Enum):
    SIGMOID = 1


class AbstractActivatorBundle:
    def sig(self, x):
        pass

    def err_gradient(self, errors, values):
        pass


class SigmoidActivatorBundle(AbstractActivatorBundle):
    def sig(self, x):
        return 1 / (1 + np.exp(-x))

    def err_gradient(self, errors, values):
        return errors * values * (1 - values)


class Pontmaster:
    weights: list[np.ndarray[typing.Any, np.dtype[np.float64]]]
    neurons: typing.Iterable[int]
    inputs: int
    outputs: int
    activator_type: ActivatorType
    activator: AbstractActivatorBundle

    def __init__(self, neurons: typing.Iterable[int], activator_type=ActivatorType.SIGMOID) -> None:
        self.neurons = neurons
        self.weights = create_weights(neurons)
        self.inputs = neurons[0]
        self.outputs = neurons[-1]
        self.activator_type = activator_type
        if activator_type == ActivatorType.SIGMOID:
            self.activator = SigmoidActivatorBundle()

    def split_training_data(self, training_data: np.ndarray):
        if len(training_data.shape) != 2:
            raise Exception(f"Training dataset must be 2-dimensional ({len(training_data.shape)}-dimensional was given)")
        if training_data.shape[-1] != self.inputs + self.outputs:
            raise Exception(f"Training data row size mismatch: {training_data.shape[-1]} instead of {self.inputs + self.outputs}")
        training_inputs = training_data[:, 0:self.inputs]
        training_outputs = training_data[:, self.inputs:]
        return training_inputs, training_outputs

    def train_repeatedly(self, training_data: np.ndarray, repetitions=10000):
        split_data = self.split_training_data(training_data)
        for i in range(repetitions):
            self.train_batch(*split_data)

    def test_one_row(self, row: np.ndarray):
        input, output = (row[0:self.inputs], row[self.inputs:])
        result = self.__propagate__(input)[-1]
        error = output - result
        return result, error

    def test_many_rows(self, data: np.ndarray):
        inputs, outputs = self.split_training_data(data)
        results = self.__propagate__(inputs)[-1]
        errors = outputs - results
        return results, errors

    def __propagate__(self, inputs: np.ndarray):
        values = [self.activator.sig(np.dot(inputs, self.weights[0]))]
        for w in self.weights[1:]:
            values.append(self.activator.sig(np.dot(values[-1], w)))
        return values

    def train_batch(self, training_inputs: np.ndarray, training_outputs: np.ndarray):
        values = self.__propagate__(training_inputs)

        gradients = [self.activator.err_gradient(training_outputs - values[-1], values[-1])]  # reversed order, output to input
        for i in range(len(self.weights)-1, 0, -1):
            gradients.append(self.activator.err_gradient(np.dot(gradients[-1], self.weights[i].T), values[i-1]))
        gradients.reverse()  # straight order, input to output

        for i in range(len(self.weights)-1, 0, -1):
            self.weights[i] += np.dot(values[i-1].T, gradients[i])
        self.weights[0] += np.dot(training_inputs.T, gradients[0])


In [2]:
training_data = np.array([
    [0, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 1, 1, 0],
    [1, 0, 0, 1],
    [1, 0, 1, 1],
    [1, 1, 0, 1],
    [1, 1, 1, 1]
])
network = Pontmaster([3, 5, 6, 1])


In [3]:

network.train_repeatedly(training_data)


In [4]:
res, errs = network.test_many_rows(training_data)
mean = errs.mean()
print("Results:")
print(res)
print("Mean error:")
print(mean)

Results:
[[0.00542405]
 [0.00204516]
 [0.00200835]
 [0.00118406]
 [0.9970519 ]
 [0.99736724]
 [0.99702747]
 [0.99700365]]
Mean error:
0.00011101475935478976
